In [ ]:
#importar las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
import seaborn as sns
import plotly.graph_objects as go
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, RobustScaler, label_binarize
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, PowerTransformer
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.tree import DecisionTreeClassifier, export_text, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')


In [ ]:
seed=12345 #fijamos la semilla de aleatorización para que sea la misma en todo el proceso
#Reemplaza con la ruta correcta y nombre de tu archivo
file_path = '/Users/luiscarrillo/Library/CloudStorage/OneDrive-Personal/Desktop/GitHub/DataScience/MachineLearning/Datasets/titanic.xlsx' 
#convertir a data frame el archivo
df = pd.read_excel(file_path)
print(df.head())
#La variable de interés es chd, binaria Si/No
#analizamos la frecuencia de cada clase
print(f'\n Instancias: {df.shape[0]}; Variables: {df.shape[1]}')
print(f'\nLa frecuencia de cada clase es: \n{df.Survived.value_counts()}')

In [ ]:
#representamos la relación enre la variable de interés y las variables input
#df['chd_numeric'] = df['Survived'].apply(lambda x: 1 if x == 'Si' else 0)
# Crear el diagrama de dispersión con regresión
sns.regplot(x=df['Title'], y=df['Survived'], ci=None,fit_reg=False)
plt.xlabel('Title ')
plt.ylabel('Survived')
plt.title('Diagrama de Dispersión')
plt.show()
# Crear el diagrama de dispersión con regresión
sns.regplot(x=df['Age'], y=df['Survived'], ci=None,fit_reg=False)
plt.xlabel('Age')
plt.ylabel('Survived')
plt.title('Diagrama de Dispersión')
plt.show()

In [ ]:
# Si se quiere categorizar la variable de respuesta (útil cuando tiene 1/0)
df['Survived_numeric'] = df['Survived']
df['Survived'] = df['Survived'].apply(lambda x: 'Yes' if x == 1 else 'No')
#en nuestro caso, cambiamos Si por Yes
#df['chd'] = df['chd'].apply(lambda x: 'Yes' if x == 'Si' else 'No')
print(df.head())

In [ ]:
# hay valores perdidos?
df.isna().sum()

Preparamos la base de datos para la aplicación de redes neuronales:

    - Estandarizar/normalizar variables continuas
    - Convertir a dummies las variables categóricas
    - Evitar missing

In [ ]:
# organiza las variables según su rol y naturaleza
# determina variable objetivo
target = "Survived"
#hacer una lista con las variables input numericas
num_cols = ['Age', 'Fare']
#hacer una lista con las variables input categóricas
cat_cols = ['Sex', 'Pclass', 'Embarked', 'Title']

In [ ]:
#Convertir a dummies las categóricas
#solo hay una variable categórica, transformación fácil
df = pd.get_dummies(df, columns=['Sex', 'Pclass', 'Embarked', 'Title'] ,drop_first=True)
# drop_first en la función get_dummies de pandas se utiliza para controlar si se debe eliminar 
#la primera columna de las variables dummy que se generan. 
#Cuando drop_first se establece en True, se elimina la primera columna de cada conjunto de variables dummy, lo que ayuda a evitar la multicolinealidad en modelos lineales
#otra opción
cat_cols = ColumnTransformer(transformers=[ ('ohe', OneHotEncoder(drop='first'), cat_cols)], 
                                                  remainder='passthrough')


In [ ]:
#Normalizar variables numericas
#Si se quisieran estandarizar, scaler=StandardScaler()
scaler = MinMaxScaler() #selecciona el transformador
X = df[num_cols] #selecciona las variables numéricas que se quieren transformar y las guarda en un nuevo dataframe
X_scale = pd.DataFrame(scaler.fit_transform(X)) #guarda el resultado de la transformación de las variables de X en X_scale
X_scale.columns = X.columns #para simplificar los nombres, asigna a las columnas de X_scale los nombres de las variables de X_num
df[num_cols] = X_scale
print(df.head())

### Otra forma de normalizar los datos para mejorar el rendimiento de la red neuronal
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [ ]:
# Separar las variables predictoras y la variable de respuesta.
# El grupo de variables predictoras se define y se fija
X = df[['Pclass_2','Fare', 'Sex_male']] #en X las variables ya están normalizadas y con dummies
y = df['Survived']
#primer approach a red neuronal: definimos la estructura
red1 = MLPClassifier(random_state=seed, hidden_layer_sizes=(8),activation='tanh',
                     alpha=0.001,solver='adam',max_iter=1000)
# Dividir los datos en entrenamiento y test (20% de los datos para test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
# Construir el modelo de red ajustando los pesos a datos de train
red1.fit(X_train, y_train)


## Si se quiere dividir en tr/v/ts
### Paso 1: División en entrenamiento y test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

### Paso 2: División del conjunto temporal en validación y test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Obtener la información sobre las capas y los coeficientes del modelo
capas = [X_train.shape[1]] + [capa.shape[0] for capa in red1.coefs_] + [len(np.unique(y_train))]

# Crear una figura de red neuronal con plotly
# Crear una figura de red neuronal con plotly
fig = go.Figure()

# Dibujar nodos y conexiones
for i in range(1, len(capas)):
    capa_anterior = capas[i - 1]
    capa_actual = capas[i]

    # Dibujar nodos
    for nodo_actual in range(capa_actual):
        fig.add_trace(go.Scatter(x=[i] * capa_actual, y=list(range(capa_actual)), mode='markers', marker=dict(size=20, color='blue')))

        # Dibujar conexiones con nodos de la capa anterior
        for nodo_anterior in range(capa_anterior):
            fig.add_trace(go.Scatter(x=[i - 1, i], y=[nodo_anterior, nodo_actual], mode='lines', line=dict(color='black')))

# Personalizar el diseño del gráfico
fig.update_layout(
    showlegend=False,
    xaxis=dict(title='Capas'),
    yaxis=dict(title='Nodos'),
    title='Estructura de la Red Neuronal',
)

# Mostrar el gráfico
fig.show()

- Probar con 200 iteraciones, qué pasa?

- Modificar la estructura de red: ¿qué cantidad de nodos ocultos es apropiada?
En este ejemplo, k=3. Como hay 469 observaciones y hemos usado tres variables input, si se reservan aproximandamente 20 observaciones para cada parámetro,la fórmula nos dice que una cantidad razonable de nodos ocultos es 5-6

In [ ]:
# ES IMPORTANTE QUE LA DISTRIBUCIÓN DE LAS CLASES SEA 'SIMILAR' EN TRAIN Y TEST.
print(f'La frecuencia de cada clase en train es: \n{y_train.value_counts(normalize=True)}')
print(f'\nLa frecuencia de cada clase en test es: \n{y_test.value_counts(normalize=True)}')

In [ ]:
# Niveles de la variable a predecir
print(red1.classes_)
# Nombre de las variables predictoras
print(red1.feature_names_in_)
# Cantidad de iteraciones necesarias para la convergencia, 
#así se puede volver a entrenar el modelo ajustando este parámetro para tener menos coste computacional
print(red1.n_iter_) #no ha hecho falta iterar tanto
#ver los coeficientes de cada enlace
# Acceder a los coeficientes (pesos) de cada capa
print(red1.coefs_)
#si se quiere ver qué atributos podemos analizar en el modelo
#dir(red1)

In [ ]:
#una vez ajustado el modelo en datos de train, lo evaluamos en datos de test
# Realizar predicciones en el conjunto de prueba
y_pred = red1.predict(X_test)

# Calcular la precisión del modelo
precision = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo en test: {precision:.4f}")

# Mostrar la matriz de confusión
matriz_confusion = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz_confusion)

# Mostrar el informe de clasificación
informe_clasificacion = classification_report(y_test, y_pred)
print("Informe de clasificación:")
print(informe_clasificacion)

In [ ]:
#otra forma de mostrar la matriz de confusión
#ConfusionMatrixDisplay, que espera etiquetas binarias
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)
y_pred_encoded = label_encoder.fit_transform(y_pred)
cm = confusion_matrix(y_test_encoded, y_pred_encoded)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusión')
plt.show()

In [ ]:
#una forma de analizar el overfitting es comparando la medida de bondad en train/test
#si hay mucha diferencia, es porque el modelo está sobreajustado
#si ambas son muy malas, es porque el modelo está poco ajustado
# Realizar predicciones en el conjunto de prueba
y_pred_tr = red1.predict(X_train)

# Calcular la precisión del modelo
precision_tr = accuracy_score(y_train, y_pred_tr)
print(f"Precisión del modelo en train: {precision_tr:.4f}")
print(f"Precisión del modelo en test: {precision:.4f}")
#este es un ejemplo de modelo sobreajustado

In [ ]:
#validación cruzada para una evaluación más robusta del modelo
#importante cambiar el tipo ed scoring atendiendo al tipo de problema
cv_scores = cross_val_score(red1, X, y, cv=5, scoring='accuracy')
cv_precision_mean = np.mean(cv_scores)

print(f'Precisión promedio mediante validación cruzada: {cv_precision_mean:.4f}')

In [ ]:
# Tuneo y evaluación predictiva del modelo para variable dependiente continua
# El grupo de variables predictoras se define y se fija
X = df[['Pclass_2','Fare', 'Sex_male']] #en X las variables ya están normalizadas y con dummies
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
red = MLPClassifier()
#definimos los parámetros que queremos tunear
params = {
    'max_iter': [600],
    'hidden_layer_sizes': [5,7,9],
    'activation': ['tanh','relu'],
    'alpha': [0.001,0.0001]
}
scoring_metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
# cv = crossvalidation con n folds con todas las combinaciones de parámetros
grid_search = GridSearchCV(estimator=red,
                           param_grid=params,
                           cv=4, scoring = scoring_metrics, refit='accuracy')

#ajusta en entrenamiento con todas las combinaciones
grid_search.fit(X_train, y_train)

In [ ]:
# Obtener resultados del grid search
results = pd.DataFrame(grid_search.cv_results_)
# Mostrar resultados
print("Resultados de Grid Search:")
print(results[['params', 'mean_test_accuracy', 'mean_test_precision_macro', 'mean_test_recall_macro', 'mean_test_f1_macro']])
#print(results) #para ver todos los atributos obtenidos y entender cómo usarlos

# Obtener el mejor modelo (en cuanto a optimización del criterio)
best_model = grid_search.best_estimator_
print(grid_search.best_estimator_)

In [ ]:
print(grid_search.best_estimator_)

In [ ]:
# se seleccionan los modelos candidatos, y analiza su robustez a lo largo de cross validation.
ac_1 = results[['split0_test_accuracy', 'split1_test_accuracy','split2_test_accuracy', 'split3_test_accuracy']].iloc[0]
ac_2 = results[['split0_test_accuracy', 'split1_test_accuracy','split2_test_accuracy', 'split3_test_accuracy']].iloc[7]
ac_3 = results[['split0_test_accuracy', 'split1_test_accuracy','split2_test_accuracy', 'split3_test_accuracy']].iloc[11]

In [ ]:
# Crear un boxplot para los cuatro valores de accuracy
plt.boxplot([ac_1.values,ac_2.values,ac_3.values], labels = ['Red0','Red7','Red11'])
plt.title('Boxplots de Accuracy para los 4 Splits')
plt.xlabel('Splits de Cross Validation')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
#analizar y reentrenar redes candidatas AUC
red1 = MLPClassifier(**results.iloc[0].params)
red7 = MLPClassifier(**results.iloc[7].params)
red10 = MLPClassifier(**results.iloc[11].params)



In [ ]:
y_auc = pd.get_dummies(y,drop_first=True) #para calcular el AUC, y debe ser numérica

X_train, X_test, y_train, y_test = train_test_split(X, y_auc, test_size=0.2, random_state=seed)



In [ ]:
# Fit the models
red1.fit(X_train, y_train)
red7.fit(X_train, y_train)
red10.fit(X_train, y_train)

# Calculamos las predicciones en test, en términos de probabilidad para poder dibujar el AUC
y_pred1 = red1.predict_proba(X_test)[:,1]
y_pred7 = red7.predict_proba(X_test)[:,1]
y_pred10 = red10.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred1)
roc_auc = auc(fpr, tpr)
print(f"\nÁrea bajo la curva ROC (AUC) para la red 1 en test: {roc_auc:.2f}")

# Graficar la curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC red1')
plt.legend(loc="lower right")
plt.show()